In [176]:
!pip install ortools

You should consider upgrading via the 'C:\Users\user\math_opt\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [177]:
from ortools.linear_solver import pywraplp

In [178]:
def flag_calculate(bins_df):
    bin_capacities = bins_df['base'].to_list()
    bin_cap_max = list()
    bin_cap_min  = list()
    
    # データの初期処理
    for i, flg in enumerate(bins_df.loc[:,"flag"]):
        if flg == 'm':
            bin_cap_max.append(bin_capacities[i] + 10000)
            bin_cap_min.append(bin_capacities[i] - 10000)
        elif flg == 'i':
            if bin_capacities[i] > 500000:
                bin_cap_min.append(bin_capacities[i] - 50000)
                bin_cap_max.append(bin_capacities[i] + 50000)
            else:
                bin_cap_min.append(bin_capacities[i] - 10000)
                bin_cap_max.append(bin_capacities[i] + 10000)
        else:
            raise ValueError("フラグの値が不正です")
    
    bins_df['max'] = bin_cap_max
    bins_df['min'] = bin_cap_min

    return bins_df

In [195]:
def make_data(bins_df:list, weights:list, values:list)->dict:

    bin_capacities = bins_df['max'].to_list()
    bin_minimumreq = bins_df['min'].to_list()
    # flagに従って、bin_capacitiesを上乗せする
    # bin_capacities
    # flagに従って、bin_capacitiesの最小値を設定する
    # bin_minimumreq


    data = {}
    data['weights'] = weights
    data['values'] = values
    assert len(data['weights']) == len(data['values'])
    data['num_items'] = len(data['weights'])
    data['all_items'] = range(data['num_items'])

    data['bin_capacities'] = bin_capacities
    data['num_bins'] = len(data['bin_capacities'])
    data['all_bins'] = range(data['num_bins'])
    return data, bin_minimumreq

In [196]:
def slv_multiknap(data,bin_minimumreq):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if solver is None:
        print('SCIP solver unavailable.')
    # x[i, b] = 1 if item i is packed in bin b.
    x = {}
    for i in data['all_items']:
        for b in data['all_bins']:
            x[i, b] = solver.BoolVar(f'x_{i}_{b}')

    # それぞれのアイテムはバッグに1つ以上入らない
    for i in data['all_items']:
        solver.Add(sum(x[i, b] for b in data['all_bins']) <= 1)

################################################################
    # それぞれのバッグにはアイテムが１つ以上入る
    for b in data['all_bins']:
        solver.Add(sum(x[i, b] for i in data['all_bins']) >= 1)
################################################################

    # キャパシティの上限・下限の設定
    for b in data['all_bins']:
        # 上限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) <= data['bin_capacities'][b])
        # 下限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) >= bin_minimumreq[b])
    #時間がかかるなら目的関数いらない
        # 個数の最大化を目標にする
        objective = solver.Objective()
        for i in data['all_items']:
            for b in data['all_bins']:
                objective.SetCoefficient(x[i, b], data['values'][i])
        objective.SetMaximization()
    

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # print(f'最大個数: {objective.Value()}\n')
        # txt = f'  最大個数: {objective.Value()}\n'
        txt = f'  最大個数: 計算無し\n'

        total_weight = 0
        for b in data['all_bins']:
            print(f'Bin {b}')
            txt += f'  Bin {b}'
            bin_weight = 0
            bin_value = 0
            for i in data['all_items']:
                if x[i, b].solution_value() > 0:
                    print(
                        f"Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    )
                    txt += f"  Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
            print(f"  目的重量範囲は: { bin_minimumreq[b] } ~ {data['bin_capacities'][b]}")
            txt += f"  目的重量範囲は: { bin_minimumreq[b] } ~ {data['bin_capacities'][b]}"
            print(f'パック内の総重量は: {bin_weight}')
            txt += f'  パック内の総重量は: {bin_weight}'
            print(f'パック内の個数は: {bin_value}\n')
            txt += f'  パック内の個数は: {bin_value}\n'
            total_weight += bin_weight
        print(f'総重量は: {total_weight}')
        txt += f'  総重量は: {total_weight}'
        with open('result.txt', 'w') as f:
            f.write(txt)
        return True
    else:
        # print('最適解なし')
        return False

In [197]:
def edakariA(bins, weights):   
    #tegより小さなクラスの削除を行う
    done = False
    while True:
        capacities = sorted(bins['min'])
        weights = sorted(weights)
    
        #小さいもの順に確認していき、
        for i in range(0,len(capacities)):
            if capacities[i] < weights[i]:

                #削除する
                bins = bins[bins["min"]!=capacities[i]]
                break
            
        print(f'i={i}  cap = {len(capacities)}')
        
        if i == len(capacities) -1:
            done = True
            break
        
        if done:
            break

    return bins

In [198]:
def  edakariC(bins, weights):
    
    # 排除２：
    # classの数と値が大きすぎて、tegが足りない

    # 方法２：
    # classとtegを大きい順に並べる

    # classの範囲内に収まるように、大きいtegから入れていく
    
    maxs = sorted(bins['max'], reverse = True)
    mins = sorted(bins['min'], reverse = True)
    weights = sorted(weights, reverse = True)

    for j in range(0,len(weights)):

        for i in range(0, len(maxs)):

            if mins[i] < 0:
                continue

            if maxs[i] >= weights[j]:
                #maxとminから差し引く
                maxs[i] = maxs[i] - weights[j]
                mins[i] = mins[i] - weights[j]

                #次のweightへ
                break

        # #classを使い切った場合
        # if i == len(maxs) - 1:

    donyokucount = i

    return donyokucount

In [199]:
def edakariB(bins, weights):

#     排除３：tegの合計値がミニマムの合計値に至らない

# 方法３：
    while True:
# 至るまで、最大値を持つクラスを削除する
        totalweights = sum(weights)
        totalmin = bins["min"].sum()

        if totalweights < totalmin:
            bins = bins[bins["min"]!=bins["min"].max()]
        else:
            break

    return bins
    

In [200]:
import pandas as pd
bins_df_original = pd.read_csv('testclass.csv')
items_df = pd.read_csv('teg.csv')
weights = items_df['list'].to_list()
values = [ 1 for x in weights]
bins = flag_calculate(bins_df_original)
bins = edakariA(bins, weights)
bins = edakariB(bins, weights)
donyokucount = edakariC(bins,weights)

i=0  cap = 30
i=0  cap = 29
i=0  cap = 28
i=26  cap = 27


In [240]:
#classの組み合わせ作成
import itertools

bins = bins.reset_index()

bins_max = len(bins)
res = False
for i in range(0,bins_max):
    bins_combs = itertools.combinations(bins.index, len(bins) - i)
    num = 0
    for bins_comb in bins_combs:
        bins_df = bins.iloc[list(bins_comb)]

        data, bin_minimumreq = make_data(bins_df,weights,values)

        res = slv_multiknap(data, bin_minimumreq)
        
        if res:
            break
        else:
            num += 1
    if res:
        break
    else:
        print(f'{i+1}回目実行 解無し 組み合わせ数　{num}個')
if res:
    print(f'{i+1}回目で処理完了')
else:
    print(f'条件を満たす結果が見つかりませんでした')

Bin 0
Item 1 重量: 440550 個数: 1
  目的重量範囲は: 435550 ~ 455550
パック内の総重量は: 440550
パック内の個数は: 1

Bin 1
Item 5 重量: 190000 個数: 1
Item 43 重量: 583000 個数: 1
  目的重量範囲は: 676000 ~ 776000
パック内の総重量は: 773000
パック内の個数は: 2

Bin 2
Item 7 重量: 122100 個数: 1
Item 45 重量: 148511 個数: 1
  目的重量範囲は: 257617 ~ 277617
パック内の総重量は: 270611
パック内の個数は: 2

Bin 3
Item 6 重量: 388245 個数: 1
Item 26 重量: 220000 個数: 1
  目的重量範囲は: 593804 ~ 693804
パック内の総重量は: 608245
パック内の個数は: 2

Bin 4
Item 14 重量: 169378 個数: 1
Item 30 重量: 308000 個数: 1
  目的重量範囲は: 457722 ~ 477722
パック内の総重量は: 477378
パック内の個数は: 2

Bin 5
Item 15 重量: 4211812 個数: 1
Item 42 重量: 971700 個数: 1
  目的重量範囲は: 5145620 ~ 5245620
パック内の総重量は: 5183512
パック内の個数は: 2

Bin 6
Item 18 重量: 671031 個数: 1
Item 35 重量: 813000 個数: 1
  目的重量範囲は: 1394340 ~ 1494340
パック内の総重量は: 1484031
パック内の個数は: 2

Bin 7
Item 21 重量: 359263 個数: 1
Item 29 重量: 450000 個数: 1
  目的重量範囲は: 768377 ~ 868377
パック内の総重量は: 809263
パック内の個数は: 2

Bin 8
Item 0 重量: 1000000 個数: 1
  目的重量範囲は: 971334 ~ 1071334
パック内の総重量は: 1000000
パック内の個数は: 1

Bin 9
Item 13 重量: 4

In [225]:
bins_combs = itertools.combinations(bins.index, len(bins) - i)
for b in bins_combs:
    print(list(b))

[0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 20, 22, 23, 26, 27, 28]


In [239]:
bins

,class2,base,flag,max,min
0,A,445550,m,455550,435550
1,B,726000,i,776000,676000
3,D,267617,i,277617,257617
4,E,643804,i,693804,593804
5,F,467722,i,477722,457722
6,G,5195620,i,5245620,5145620
7,H,1444340,i,1494340,1394340
8,I,818377,i,868377,768377
10,K,1021334,i,1071334,971334
11,L,412366,i,422366,402366
